In [2]:
!pip install -q imdb-sqlite

In [3]:
!pip install -q pycountry

In [5]:
# Importando pacotes necessários
import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style = "whitegrid")